<a href="https://colab.research.google.com/github/Danieljoshua720/Electricity-demand-forecasting-AIML/blob/main/lstm_code_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------
# 1️⃣ Import Libraries
# ----------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ----------------------------
# 2️⃣ Load Dataset
# ----------------------------
df = pd.read_csv("electricity_demand.csv", parse_dates=["Datetime"])
df.set_index("Datetime", inplace=True)

# Optional: add features
df['Hour'] = df.index.hour
df['DayOfWeek'] = df.index.dayofweek

# Keep target + features
target_column = "Demand"
feature_columns = ["Demand", "Hour", "DayOfWeek"]  # Multivariate

data = df[feature_columns].values

# ----------------------------
# 3️⃣ Scale Data
# ----------------------------
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# ----------------------------
# 4️⃣ Sequence Creation
# ----------------------------
def create_sequences(data, lookback=24, horizon=1, target_idx=0):
    X, y = [], []
    for i in range(lookback, len(data) - horizon + 1):
        X.append(data[i-lookback:i])
        y.append(data[i:i+horizon, target_idx])
    return np.array(X), np.array(y)

LOOKBACK = 24   # past 24 hours
HORIZON = 1     # predict 1 step ahead
TARGET_IDX = 0  # Demand column

X, y = create_sequences(data_scaled, lookback=LOOKBACK, horizon=HORIZON, target_idx=TARGET_IDX)

# ----------------------------
# 5️⃣ Train/Test Split
# ----------------------------
train_ratio = 0.8
train_size = int(len(X) * train_ratio)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print("X_train shape:", X_train.shape)
print("X_test shape :", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape :", y_test.shape)

# ----------------------------
# 6️⃣ Build LSTM Model
# ----------------------------
def build_lstm_model(input_shape, units=64, layers=1, dropout=0.0):
    model = Sequential()
    for i in range(layers):
        return_sequences = i < layers - 1
        model.add(LSTM(units, return_sequences=return_sequences, input_shape=input_shape if i==0 else None))
        if dropout > 0:
            model.add(Dropout(dropout))
    model.add(Dense(HORIZON))
    model.compile(optimizer='adam', loss='mse')
    return model

model = build_lstm_model(input_shape=(LOOKBACK, X_train.shape[2]), units=64, layers=1, dropout=0.1)
model.summary()

# ----------------------------
# 7️⃣ Train Model with Callbacks
# ----------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
    ModelCheckpoint('best_lstm_model.h5', save_best_only=True)
]

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# ----------------------------
# 8️⃣ Predictions & Inverse Transform
# ----------------------------
y_train_pred = model.predict(X_train)
y_test_pred  = model.predict(X_test)

# Only inverse transform the target column (first column)
y_train_pred_inv = scaler.inverse_transform(
    np.hstack([y_train_pred, np.zeros((y_train_pred.shape[0], data.shape[1]-1))])
)[:,0]

y_train_inv = scaler.inverse_transform(
    np.hstack([y_train, np.zeros((y_train.shape[0], data.shape[1]-1))])
)[:,0]

y_test_pred_inv = scaler.inverse_transform(
    np.hstack([y_test_pred, np.zeros((y_test_pred.shape[0], data.shape[1]-1))])
)[:,0]

y_test_inv = scaler.inverse_transform(
    np.hstack([y_test, np.zeros((y_test.shape[0], data.shape[1]-1))])
)[:,0]

# ----------------------------
# 9️⃣ Evaluate Metrics
# ----------------------------
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred)/y_true)) * 100
    return mae, rmse, mape

train_mae, train_rmse, train_mape = evaluate(y_train_inv, y_train_pred_inv)
test_mae, test_rmse, test_mape = evaluate(y_test_inv, y_test_pred_inv)

print(f"Train MAE : {train_mae:.2f}, RMSE: {train_rmse:.2f}, MAPE: {train_mape:.2f}%")
print(f"Test  MAE : {test_mae:.2f}, RMSE: {test_rmse:.2f}, MAPE: {test_mape:.2f}%")

# ----------------------------
# 🔟 Visualization
# ----------------------------
plt.figure(figsize=(12,5))
plt.plot(df.index[-len(y_test_inv):], y_test_inv, label='Actual')
plt.plot(df.index[-len(y_test_pred_inv):], y_test_pred_inv, label='Predicted')
plt.title("LSTM Forecast – Electricity Demand")
plt.xlabel("Time")
plt.ylabel("Demand")
plt.legend()
plt.show()
